In [ ]:
# # load autoreloader
# %reload_ext autoreload
# %load_ext tensorboard
# %autoreload 2

In [1]:
# import memory_gym
import gymnasium as gym, os, json

os.environ["DISPLAY"] = ":0"
from stable_baselines3 import PPO
from tqdm.notebook import tqdm
from stable_baselines3.common.logger import Logger, HumanOutputFormat
from stable_baselines3.common.vec_env.base_vec_env import (
    VecEnv,
    VecEnvStepReturn,
    VecEnvWrapper,
)
from stable_baselines3.common.policies import ActorCriticPolicy, MultiInputActorCriticPolicy
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecNormalize
from stable_baselines3.common.logger import TensorBoardOutputFormat, configure
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3.common.callbacks import BaseCallback

os.sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from sb3 import model as model_lib

from sb3 import helm as helm_lib

/common/home/sj1030/miniconda3/envs/memory-gym/lib/python3.11/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
/common/home/sj1030/miniconda3/envs/memory-gym/lib/python3.11/site-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/common/home/sj1030/miniconda3/envs/memory-gym/lib/python3.11/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
2023-11-20 19:12:00.182136: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Ten

In [2]:
env = gym.make(
    "memory_gym:MortarMayhem-v0",
)
json.loads(env.spec.to_json())

# # env = gym.make('MortarMayhem-v0', )
# env_func = lambda: gym.make('MortarMayhem-v0', )
# # envs = [gym.make('MortarMayhem-v0') for _ in range(4)]
# env = VecNormalize(DummyVecEnv([env_func]*4))

# # json.loads(env.spec.to_json())

{'id': 'MortarMayhem-v0',
 'entry_point': 'memory_gym.mortar_mayhem:MortarMayhemEnv',
 'reward_threshold': None,
 'nondeterministic': False,
 'max_episode_steps': None,
 'order_enforce': True,
 'autoreset': False,
 'disable_env_checker': False,
 'apply_api_compatibility': False,
 'kwargs': {},
 'additional_wrappers': [],
 'vector_entry_point': None}

In [3]:
env.observation_space, env.action_space

(Box(0.0, 1.0, (84, 84, 3), float32), MultiDiscrete([3 3]))

In [4]:
class EpisodicRewardCallback(BaseCallback):
    def __init__(self, verbose=0, plot_freq=10, log_dir="./tensorboard_logs/"):
        super(EpisodicRewardCallback, self).__init__(verbose)
        self.episode_rewards = []
        self.plot_freq = plot_freq
        self.log_dir = log_dir

    def on_rollout_start(self) -> None:
        # print("starting rollout")
        self.ep_rewards = 0.0

    def _on_step(self):
        self.ep_rewards += self.locals["rewards"][-1]

    def _on_rollout_end(self) -> None:
        # print("ending rollout")
        episode_reward = self.ep_rewards
        self.episode_rewards.append(episode_reward)
        # print(f"Episode reward: {episode_reward}")

    # def _on_training_end(self) -> None:
    #     self._plot_rewards()

    # def _plot_rewards(self):
    #     x = np.arange(1, len(self.episode_rewards) + 1)
    #     plt.figure(figsize=(10, 5))
    #     plt.plot(x, self.episode_rewards)
    #     plt.xlabel('Episode')
    #     plt.ylabel('Total Reward')
    #     plt.title('Episodic Rewards')
    #     plt.grid(True)
    #     plt.savefig(os.path.join(self.log_dir, 'episodic_rewards.png'))
    #     plt.show()

In [5]:
def linear_schedule(initial_lr, final_lr, total_timesteps):
    def schedule(progress):
        fraction = min(progress / total_timesteps, 1.0)
        return initial_lr + fraction * (final_lr - initial_lr)

    return schedule


def custom_entropy_schedule(initial_entropy_coef, final_entropy_coef, total_timesteps):
    def entropy_coef_schedule(progress):
        fraction = min(progress / total_timesteps, 1.0)
        return torch.tensor(
            initial_entropy_coef
            + fraction * (final_entropy_coef - initial_entropy_coef),
            dtype=torch.float32,
        )

    return entropy_coef_schedule

In [6]:
model = helm_lib.HELMPPO(
    MultiInputActorCriticPolicy,
    env,
    verbose=2,
    # n_steps=total_timesteps,
    n_epochs=3,
    vf_coef=0.5,
    clip_range=0.2,
    batch_size=16384,
    max_grad_norm=0.5,
    gae_lambda=0.95,
    gamma=0.99,
    # ent_coef=custom_entropy_schedule(1e-4, 1e-5, total_timesteps),
    seed=1234,
    tensorboard_log="/common/home/sj1030/Projects/memory-gym/sb3/tensorboard_logs/",
    clip_decay="none",
    _init_setup_model=False,
)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/common/home/sj1030/Projects/memory-gym/sb3/helm.py:162: UserWarning: You have specified a mini-batch size of 16384, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2048`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 2048
We recommend using a `batch_size` that is a multiple of `n_steps * n_envs`.
Info: (n_steps=2048 and n_envs=1)
  warnings.warn(


torch.Size([1, 3, 84, 84])


In [7]:
# callback = EpisodicRewardCallback(plot_freq=10, log_dir="tensorboard_logs/")

model.learn(total_timesteps=1000, progress_bar=True)

Logging to /common/home/sj1030/Projects/memory-gym/sb3/tensorboard_logs/HELM_14


Output()

In [ ]:
# callback.plot_rewards()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
arr = np.load("rewards.npy")
df = pd.DataFrame(arr, columns=["rewards"])

In [ ]:
TSBOARD_SMOOTHING = [0.5, 0.85, 0.99]
ts_factor = 0.99
# for ts_factor in TSBOARD_SMOOTHING:
smooth = df[2].ewm(alpha=(1 - ts_factor)).mean()
plt.figure(figsize=(13, 5))
# for ptx in range(3):
# plt.subplot(1,3,ptx+1)
plt.plot(df[2], alpha=0.1)
plt.plot(smooth)
plt.title("Mean Episodic Length", fontdict={"fontsize": 20})
plt.grid(alpha=0.3)
plt.ylim(58.5, 64)

label_interval = 200
x_tick_positions = np.arange(0, len(df), label_interval)
x_tick_labels = (x_tick_positions * 2e4).astype(int)

# Set custom x-tick labels
plt.xticks(ticks=x_tick_positions, labels=x_tick_labels)

plt.show()